In [3]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import vectorbt as vbt

from backtest.use_cases.backtest_execute import backtest_execute
from backtest.use_cases.strategy_execute import (stocastic_rsi_ema_mix_function,
                                                 trendilo_mix_function,
                                                 ema_local_min_max_trandline_function,
                                                 ema_swing_low_function,
                                                 candle_stick_pattern,
                                                 ema_swing_another_function,
                                                 finally_fib,
                                                 swing_search,)
from backtest.domains.strategy_result import StrategyResultColumnType
from backtest.request.stockdata_from_repo import build_stock_data_from_repo_request
from backtest.use_cases.stockdata_from_repo import stockdata_from_repo
from backtest.repository.webrepo.crypto.upbit_repo import UpbitRepo
from backtest.domains.backtest_plot_package import BacktestPlotPackage
from backtest.domains.backtest import Backtest
from backtest.domains.strategy import Strategy
from backtest.domains.stockdata import StockData
from backtest.use_cases.standardize_stock import standardize_stock
from backtest.use_cases.make_crypto_backtest import get_upbit_symbol
from datetime import datetime, timedelta
import time
import pyupbit
from typing import List

def build_support_stockdata(stockdata_list : List[StockData]) -> pd.DataFrame:
    standardize_stock(stockdata_list)
    stockdata_support_df = pd.DataFrame(index=stockdata_list[0].data.index)
    for stockdata in stockdata_list:
        stockdata_df = stockdata.data
        symbol = stockdata.symbol
        stockdata_df['krw'] = stockdata_df['close'] * stockdata_df['volume']
        stockdata_support_df[symbol] = stockdata_df['krw'].rolling(48).sum()
    result_support_df = stockdata_support_df.rank(axis=1,ascending = 0)    
    
    return result_support_df

def build_stockdata(symbol: str, from_date: str, cache: bool = False,check_rank: bool = False, big_stock: bool = False):
    request = build_stock_data_from_repo_request(
        filters={'order__eq': symbol, 'from__eq': from_date, 'chart_interval__eq':'240m'})
    response = stockdata_from_repo(UpbitRepo(), request=request, cache=cache)
    stockdata = response.value
    support_df = None
    support_symbols=[ symbol.replace("KRW-","") for symbol in pyupbit.get_tickers(fiat="KRW")]
    support_stockdata = []
    if check_rank:
        for idx,support_symbol in enumerate(support_symbols,1):
            # print("REQUEST SYMBOL : {} [{}/{}]".format(support_symbol,idx,len(support_symbols)))
            request2 = build_stock_data_from_repo_request(
            filters={'order__eq': support_symbol, 'from__eq': from_date})
            response2 = stockdata_from_repo(UpbitRepo(), request=request2, cache=cache)
            stockdata2 =response2.value
            support_stockdata.append(stockdata2)
            support_df = build_support_stockdata(support_stockdata)
    big_stockdata = None
    if big_stock:
        big_stock_from_date = datetime.strptime(from_date,'%Y-%m-%d') - timedelta(days=900)
        big_request = build_stock_data_from_repo_request(
            filters={'order__eq': symbol, 'from__eq': big_stock_from_date.strftime('%Y-%m-%d')})
        big_response = stockdata_from_repo(UpbitRepo(), request=big_request, cache=cache)
        big_stockdata = big_response.value

    return stockdata , support_df , big_stockdata

symbols=[symbol.replace("KRW-","") for symbol in pyupbit.get_tickers(fiat="KRW")]
symbols=['MTL']
for symbol in symbols:
    from_date = datetime.now() - timedelta(days=200)
    try:
        from_date_str = from_date.strftime("%Y-%m-%d")
        stockdata,support_df,big_stockdata = build_stockdata(symbol=symbol, from_date=from_date_str, cache=False, big_stock=False, check_rank=False)
        #strategy = Strategy(name='stocastic_rsi_ema_mix_function', function=stocastic_rsi_ema_mix_function, weight=100,  options={'heikin_ashi':{stockdata.symbol: stockdata2},'compare_movement':1})
        function_name_list = ['finally_fib','swing_search']
        strategy1 = Strategy(name='finally_fib',function=finally_fib,weight=100,options={'support_df':support_df,'vp_range':250})
        strategy2 = Strategy(name='swing_search',function=swing_search,weight=100,options={'support_df':support_df,})
        #strategy4 =  Strategy(name='candle_stick_pattern',function=candle_stick_pattern,weight=100,options={'big_stockdata':big_stockdata,'rolling':30})
        backtest = Backtest(strategy_list=[strategy1,strategy2], stockdata_list=[stockdata])
        plot_package = BacktestPlotPackage()
        backtest_execute(
            backtest, verbose=False, plot_package=plot_package).value
        for function_name,strategy_result in zip(function_name_list, plot_package.package_data_bucket[symbol]):
            strategy_result = strategy_result[function_name][function_name]
            buy_signal = strategy_result.apply(lambda r : True if r[0] == StrategyResultColumnType.BUY else False)
            sell_signal = strategy_result.apply(lambda r : True if r[0] == StrategyResultColumnType.SELL else False)
            print('function result :',function_name)
            pf = vbt.Portfolio.from_signals(stockdata.data.close,buy_signal,sell_signal,fees=0.0005)
            result=pf.stats()
            print(symbol,result['Win Rate [%]'],result['Total Return [%]'])
        time.sleep(5)
    except Exception as e:
        print('error',symbol)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[WARNING] could not import module backtest.module_compet.talib.talib trying next..
[INFO] using backtest.module_compet.talib.pandas for calculate strategy


ModuleNotFoundError: No module named 'talib'